In [1]:
import pandas as pd

# Load the main dataset
weather_data = pd.read_csv(r'C:\Users\Admin\OneDrive\Documents\Desktop\New Folder\archive\weather_prediction_dataset.csv')

# Load the label dataset (BBQ labels)
labels_data = pd.read_csv(r'C:\Users\Admin\OneDrive\Documents\Desktop\New Folder\archive\weather_prediction_bbq_labels.csv')

In [2]:
# Check for common columns
print(weather_data.columns)
print(labels_data.columns)

Index(['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_humidity',
       'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation',
       'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min',
       ...
       'STOCKHOLM_temp_min', 'STOCKHOLM_temp_max', 'TOURS_wind_speed',
       'TOURS_humidity', 'TOURS_pressure', 'TOURS_global_radiation',
       'TOURS_precipitation', 'TOURS_temp_mean', 'TOURS_temp_min',
       'TOURS_temp_max'],
      dtype='object', length=165)
Index(['DATE', 'BASEL_BBQ_weather', 'BUDAPEST_BBQ_weather', 'DE_BBQ_weather',
       'DRESDEN_BBQ_weather', 'DUSSELDORF_BBQ_weather', 'HEATHROW_BBQ_weather',
       'KASSEL_BBQ_weather', 'LJUBLJANA_BBQ_weather', 'MAASTRICHT_BBQ_weather',
       'MALMO_BBQ_weather', 'MONTELIMAR_BBQ_weather', 'MUENCHEN_BBQ_weather',
       'OSLO_BBQ_weather', 'PERPIGNAN_BBQ_weather', 'SONNBLICK_BBQ_weather',
       'STOCKHOLM_BBQ_weather', 'TOURS_BBQ_weather'],
      dtype='object')


In [3]:
# Merge the datasets on the common column (replace 'Date' with the actual column name)
merged_data = pd.merge(weather_data, labels_data, on='DATE')
# Check the merged data structure
print(merged_data.head())

       DATE  MONTH  BASEL_cloud_cover  BASEL_humidity  BASEL_pressure  \
0  20000101      1                  8            0.89          1.0286   
1  20000102      1                  8            0.87          1.0318   
2  20000103      1                  5            0.81          1.0314   
3  20000104      1                  7            0.79          1.0262   
4  20000105      1                  5            0.90          1.0246   

   BASEL_global_radiation  BASEL_precipitation  BASEL_sunshine  \
0                    0.20                 0.03             0.0   
1                    0.25                 0.00             0.0   
2                    0.50                 0.00             3.7   
3                    0.63                 0.35             6.9   
4                    0.51                 0.07             3.7   

   BASEL_temp_mean  BASEL_temp_min  ...  LJUBLJANA_BBQ_weather  \
0              2.9             1.6  ...                  False   
1              3.6             2

Weather Parameters: These columns reflect the conditions that influence whether it's suitable for a BBQ. Key features include:

#Temperature:
BASEL_temp_mean
BASEL_temp_min
BASEL_temp_max

#Humidity:
BASEL_humidity

#Pressure:
BASEL_pressure

#Precipitation:
BASEL_precipitation
Wind Speed:
TOURS_wind_speed

#Sunshine Duration:
BASEL_sunshine
Cloud Cover:
BASEL_cloud_cover

#Global Radiation:
BASEL_global_radiation

In [4]:
print(merged_data.columns)


Index(['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_humidity',
       'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation',
       'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min',
       ...
       'LJUBLJANA_BBQ_weather', 'MAASTRICHT_BBQ_weather', 'MALMO_BBQ_weather',
       'MONTELIMAR_BBQ_weather', 'MUENCHEN_BBQ_weather', 'OSLO_BBQ_weather',
       'PERPIGNAN_BBQ_weather', 'SONNBLICK_BBQ_weather',
       'STOCKHOLM_BBQ_weather', 'TOURS_BBQ_weather'],
      dtype='object', length=182)


In [ ]:
# Check for missing values
print(merged_data.isnull().sum())

# Visualize missing data using a heatmap
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(merged_data.isnull(), cbar=False, cmap='viridis')
plt.show()

In [5]:
# Check the available columns in the dataset
print(merged_data.columns)

# Fill missing numerical values with the mean for the correct columns
merged_data['BASEL_temp_mean'] = merged_data['BASEL_temp_mean'].fillna(merged_data['BASEL_temp_mean'].mean())
merged_data['BASEL_humidity'] = merged_data['BASEL_humidity'].fillna(merged_data['BASEL_humidity'].mean())
merged_data['TOURS_wind_speed'] = merged_data['TOURS_wind_speed'].fillna(merged_data['TOURS_wind_speed'].mean())

Index(['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_humidity',
       'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation',
       'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min',
       ...
       'LJUBLJANA_BBQ_weather', 'MAASTRICHT_BBQ_weather', 'MALMO_BBQ_weather',
       'MONTELIMAR_BBQ_weather', 'MUENCHEN_BBQ_weather', 'OSLO_BBQ_weather',
       'PERPIGNAN_BBQ_weather', 'SONNBLICK_BBQ_weather',
       'STOCKHOLM_BBQ_weather', 'TOURS_BBQ_weather'],
      dtype='object', length=182)


In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Replace 'Rain' with the correct column name
merged_data['BASEL_precipitation_encoded'] = label_encoder.fit_transform(merged_data['BASEL_precipitation'])

In [15]:
X = merged_data[['BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max', 
                 'BASEL_humidity', 'BASEL_pressure', 'BASEL_precipitation', 
                 'TOURS_wind_speed', 'BASEL_sunshine', 'BASEL_cloud_cover', 
                 'BASEL_global_radiation', 'BUDAPEST_temp_mean', 
                 'STOCKHOLM_temp_mean']]  # Include more city data if relevant

y = merged_data[['BASEL_BBQ_weather', 'BUDAPEST_BBQ_weather', 'DRESDEN_BBQ_weather']]  # Adjust based on your focus


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Training set size: {X_train.shape}')
print(f'Testing set size: {X_test.shape}')

Training set size: (2923, 12)
Testing set size: (731, 12)


In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

# Instantiate the model
model = GaussianNB()

# Train the model (use one target variable for demonstration)
model.fit(X_train, y_train['BASEL_BBQ_weather'])

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test['BASEL_BBQ_weather'], y_pred))


              precision    recall  f1-score   support

       False       1.00      0.96      0.98       542
        True       0.89      0.99      0.94       189

    accuracy                           0.97       731
   macro avg       0.94      0.97      0.96       731
weighted avg       0.97      0.97      0.97       731

